In [3]:
from tensorflow import keras

m = 32
model = keras.Sequential([
    keras.Input(shape=(4,)),
    keras.layers.Dense(m, activation='relu'),  # 2^n
    keras.layers.Dense(m, activation='relu'),  # 2^n
    keras.layers.Dense(m, activation='relu'),  # 2^n
    keras.layers.Dense(m, activation='relu'),  # 2^n
    keras.layers.Dense(m, activation='relu'),  # 2^n
    keras.layers.Dense(3, activation='sigmoid')])  # sigmoid a verifier

model.compile(optimizer='adam', loss='categorical_crossentropy',
              metrics=['accuracy'])  #try d'autre loss



In [2]:
import os
import pandas as pd
import math
import networkx as nx
from python_tsp.exact import solve_tsp_dynamic_programming
import numpy as np

def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # rayon de la Terre en km
    lat1, lon1, lat2, lon2 = map(math.radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = math.sin(dlat / 2) ** 2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2) ** 2
    c = 2 * math.asin(math.sqrt(a))
    return R * c


def create_distance_matrix(data):
    n = len(data)
    matrix = np.zeros((n, n))
    for i, row1 in enumerate(data.itertuples()):
        for j, row2 in enumerate(data.itertuples()):
            if i != j:
                matrix[i][j] = haversine(row1.latitude, row1.longitude, row2.latitude, row2.longitude)
    return matrix


# Chemin du dossier contenant les fichiers CSV
data_folder = "dataTest"


# Parcourir tous les fichiers du dossier
for file in os.listdir(data_folder):
    # Vérifier si le fichier est un fichier CSV
    if file.endswith(".csv"):
        # Construire le chemin complet du fichier
        file_path = os.path.join(data_folder, file)

        # Lire le fichier CSV et le convertir en dataframe
        df = pd.read_csv(file_path)
        # Print tout les differents lignes
        print(df[" nomcourtligne"].unique())

        # Filtrer les lignes en fonction de la condition nomcourtligne == 'C1' et sens == 0
        filtered_df = df[df[" nomcourtligne"] == " 'C1'"]
        filtered_df = filtered_df[filtered_df["sens"] == 0]

        # Extraire les coordonnées et les diviser en latitude et longitude
        filtered_df['latitude'] = filtered_df['coordonnees'].apply(
            lambda x: float(x.replace("[", "").split(";")[0].strip('[]')))
        filtered_df['longitude'] = filtered_df['coordonnees'].apply(
            lambda x: float(x.replace("]", "").split(";")[1].strip('[]')))


        # Create a distance matrix from the data
        distance_matrix = create_distance_matrix(filtered_df)

        # Trouver la solution approximative du TSP en utilisant l'algorithme de Christofides
        tsp_solution = solve_tsp_dynamic_programming(distance_matrix)
        # Trier le dataframe en fonction de l'ordre des bus dans la solution TSP
        ordered_df = filtered_df.iloc[tsp_solution[0]]

        distances = []
        for i in range(len(filtered_df) - 1):
            lat1 = filtered_df.iloc[i]['latitude']
            lon1 = filtered_df.iloc[i]['longitude']
            lat2 = filtered_df.iloc[i + 1]['latitude']
            lon2 = filtered_df.iloc[i + 1]['longitude']
            distance = haversine(lat1, lon1, lat2, lon2)
            distances.append(distance)

        # Calculer l'écart de distance moyen entre les bus
        if distances:
            avg_distance = sum(distances) / len(distances)
        else:
            avg_distance = None

        # Obtenir le nombre de bus
        bus_count = len(filtered_df)
        # Créer un nouveau dataframe avec l'écart de distance moyen et le nombre de bus
        data = {'avg_distance': [avg_distance], 'bus_count': [bus_count]}
        summary_df = pd.DataFrame(data, columns=['avg_distance', 'bus_count'])

        print(summary_df)





# model.fit(X_train, y_train, epochs=500, validation_data=(X_test, y_test))

[" '10'" " 'C5'" " '32'" " '12'" " '11'" " '14'" " '64'" " '13'" " 'C4'"
 " 'C6'" " 'C1'" " 'C2'" " 'C3'" " '53'" " '50'" " '75'" " '73'" " '67'"
 " '78'" " '77'" " '71'" " '56'" " '59'" " '61'" " 'C7'" " '55'" " '54'"
 " '52'" " '51'" " '62'" " '65'" " '76'" " '72'" " '80'" " '81'" " '82'"
 " '74'" " '68'"]
   avg_distance  bus_count
0      3.637694          6
